In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab08.ipynb")

# Lab 8: Model Selection, Regularization, and Cross-Validation
In this lab, you will practice using `scikit-learn` to generate models of various complexity. You'll then use the holdout method and K-fold cross-validation to select the models that generalize the best.

The on-time deadline is **Tuesday, October 24th 11:59 PM PT**. Please read the syllabus for the grace period policy. No late submissions beyond the grace period will be accepted.

### Collaboration Policy
Data science is a collaborative activity. While you may talk with others about this assignment, we ask that you **write your solutions individually**. If you discuss the assignment with others, please **include their names** in the cell below.

**Collaborators:** *list names here*

### Lab Walk-Through
In addition to the lab notebook, we have also released a prerecorded walk-through video of the lab. We encourage you to reference this video as you work through the lab. Run the cell below to display the video.

**Note:** Some part of the video is recorded in Spring 2022. There may be slight inconsistencies between the version you are viewing and the version used in the recording, but content is identical.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("uQ3E4pejmD8", list = 'PLQCcNQgUcDfqlXYhOXB-ZiE9V-4v-DU6I', listType = 'playlist')

In [ ]:
# Run this cell to set up your notebook; no further action is needed.
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
sns.set_context("talk")

from IPython.display import display, Latex, Markdown

<br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

### Introduction

For this lab, we will use a toy dataset to predict the house prices in Boston with data provided by the `sklearn.datasets` package. There are more interesting datasets in the package if you want to explore them during your free time!

Run the following cell to load the data. `pickle.load()` will return a dictionary object which includes keys for:
- `data` : The independent variables/features (X),
- `target` : The response vector (Y),
- `feature_names`: The column names,
- `DESCR` : A full description of the data, and
- `filename`: The name of the CSV file.


In [ ]:
import pickle
boston_data = pickle.load(open("boston_data.pickle", "rb")) 

print("Dictionary keys:")
print(boston_data.keys())

print()

print("Sum of the attributes:")
print(sum(boston_data.data))

In [ ]:
print(boston_data['DESCR'])

A look at the `DESCR` attribute tells us the data contains these features:

    1. CRIM      per capita crime rate by town
    2. ZN        proportion of residential land zoned for lots over 
                 25,000 sq.ft.
    3. INDUS     proportion of non-retail business acres per town
    4. CHAS      Charles River dummy variable (= 1 if tract bounds 
                 river; 0 otherwise)
    5. NOX       nitric oxides concentration (parts per 10 million)
    6. RM        average number of rooms per dwelling
    7. AGE       proportion of owner-occupied units built prior to 1940
    8. DIS       weighted distances to five Boston employment centres
    9. RAD       index of accessibility to radial highways
    10. TAX      full-value property-tax rate per 10,000 USD
    11. PTRATIO  pupil-teacher ratio by town
    12. LSTAT    % lower status of the population
    
Let's now convert this data into a `pandas` `DataFrame`. 

In [ ]:
boston = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
boston.head()

<br>

---

### Question 1

Let's model this housing price data! Before we can do this, however, we need to split the data into training and validation sets. Remember that the response vector (housing prices) lives in the `target` attribute. A random seed is set here so we can deterministically generate the same splitting in the future if we want to test our result again and find potential bugs.

Use the sklearn's `train_test_split` [(documentation)](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to split out 10% of the data for the validation set. Call the resulting splits `X_train`, `X_validation`, `Y_train`, and `Y_validation`.

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(45)

X = boston
Y = pd.Series(boston_data['target'])

X_train, X_validation, Y_train, Y_validation = ...

In [ ]:
grader.check("q1")

<br>

---

### Question 2a

As a warmup, fit a linear model to describe the relationship between the housing price and all available independent variables/features. We have imported `sklearn.linear_model` as `lm`, so you can use that instead of typing out the whole module name. Fill in the cell below to fit a model using the data contained in the training set. Then, use this fitted model to predict the housing prices for the holdout set. We've written out the code to create a scatter plot showing the relationship between the predicted and actual housing prices in the holdout set for you. 

In [ ]:
import sklearn.linear_model as lm

linear_model = lm.LinearRegression()

# Fit your linear model
# linear_model.fit(...)

# Predict housing prices on the validation set
Y_pred = ...

# DO NOT CHANGE THE CODE BELOW THIS LINE
# Plot true prices vs. predicted values
plt.scatter(Y_validation, Y_pred, alpha=0.5)

# Plot the x=y diagonal line
plt.plot([0, 50], [0, 50], color='r')
plt.xlabel("Prices $(y)$")
plt.ylabel("Predicted Prices $(\hat{y})$")
plt.title("Prices vs. Predicted Prices");

In [ ]:
grader.check("q2a")

<br>

---

### Question 2b

Briefly analyze the scatter plot above. Do you notice any outliers? Write your answer in the cell below.

_Type your answer here, replacing this text._

Alternatively, we can plot the residuals against our model predictions (fitted values). This is known as the **residual plot**. Ideally, they would all be zero. Given the inevitability of noise, we would at least like them to be scattered randomly across the line where the residual is zero. By contrast, there appears to be a possible pattern with our model consistently underestimating prices for both very low and very high values, and possibly consistently overestimating prices towards the middle range below. We can figure out whether the model is overestimating or underestimating based on the sign of the residual: if it is negative, it means $y_i < \hat{y}_i$ so we are overestimating prices. Conversely, if the residual is positive, $y_i > \hat{y_i}$ and we end up underestimating prices.

In [ ]:
plt.scatter(Y_pred, Y_validation - Y_pred, alpha=0.5)
plt.ylabel("Residual $(y - \hat{y})$")
plt.xlabel("Predicted Prices $(\hat{y})$")
plt.title("Residuals vs. Predicted Prices")
plt.title("Residual of prediction for i-th house")
plt.axhline(y = 0, color='r');

<br>

---

### Question 3a

As we see in the scatter plots above, our model is not perfect. If it were perfect, the first scatter plot of predicted vs. true prices would follow the identity line (i.e., a line of slope 1 or $\hat{y} = y$). In contrast, the second scatter plot of residuals vs. predicted prices would be scattered randomly around the line where the residual is 0. To quantify the performance of the model, we will compute the Root Mean Squared Error (RMSE) of the predicted responses: 

$$
\textbf{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^n \left( y_i - \hat{y}_i \right)^2 }
$$

Fill in the function below to compute the RMSE. Then, assign `train_error` to the model’s RMSE when making predictions on the training data and `validation_error` to the model’s RMSE when making predictions on the validation set. Your implementation **should not** use `for` loops.

In [ ]:
def rmse(actual_y, predicted_y):
    """
    Args:
        predicted_y: An array of the predictions from the model.
        actual_y: An array of the ground truth labels.
        
    Returns:
        The root mean square error between the predictions and ground truth labels.
    """
    ...

train_error = ...
validation_error = ...

print("Training RMSE:", train_error)
print("Validation RMSE:", validation_error)

In [ ]:
grader.check("q3a")

<br>

---

### Question 3b

Is your training error lower than the error on the validation set, which includes data the model never got to see while it was being trained? If so, why could this be happening? Answer in the cell below.


_Type your answer here, replacing this text._

<br>
<hr style="border: 1px solid #fdb515;" />

## Overfitting

Sometimes we can get even higher accuracy by adding more features. For example, the code below adds the square, square root, and hyperbolic tangent of every feature to the design matrix. We've chosen these bizarre features specifically to highlight overfitting.

In [ ]:
boston_with_extra_features = boston.copy()
for feature_name in boston.columns:
    boston_with_extra_features[feature_name + "^2"] = boston_with_extra_features[feature_name] ** 2
    boston_with_extra_features["sqrt" + feature_name] = np.sqrt(boston_with_extra_features[feature_name])
    boston_with_extra_features["tanh" + feature_name] = np.tanh(boston_with_extra_features[feature_name])
    
boston_with_extra_features.head(5)

We split up our data again and refit the model. From this cell forward, we append `2` to the variable names `X_train, X_validation, Y_train, Y_validation, train_error, validation_error` in order to maintain our original data.

In [ ]:
np.random.seed(25)
X = boston_with_extra_features
X_train2, X_validation2, Y_train2, Y_validation2 = train_test_split(X, Y, test_size = 0.1)
linear_model.fit(X_train2, Y_train2);

Now, let's see the training and validation RMSE values:

In [ ]:
train_error2 = rmse(Y_train2, linear_model.predict(X_train2)) 
validation_error2 = rmse(Y_validation2, linear_model.predict(X_validation2))

print("Training RMSE:", train_error2)
print("Validation RMSE:", validation_error2)

Looking at our training and validation RMSE, we see that they are lower than you computed earlier. This strange model is seemingly better even though it includes seemingly useless features like the hyperbolic tangent of the average number of rooms per dwelling.

The code below generates the training and validation RMSE for 49 different models and stores the results in a `DataFrame`. The first model uses only the first feature "CRIM". The second model uses the first two features "CRIM" and "ZN", and so forth.

In [ ]:
errors_vs_N = pd.DataFrame(columns = ["N", "Training Error", "Validation Error"])

range_of_num_features = range(1, X_train2.shape[1] + 1)

# Iterates through different number of features
for N in range_of_num_features:
    # Use only the first N features for this model
    X_train_first_N_features = X_train2.iloc[:, :N]    
    linear_model.fit(X_train_first_N_features, Y_train2)
    train_error_overfit = rmse(Y_train2, linear_model.predict(X_train_first_N_features))
    
    # Preprocess validation set the same way as the training data
    X_validation_first_N_features = X_validation2.iloc[:, :N]
    validation_error_overfit = rmse(Y_validation2, linear_model.predict(X_validation_first_N_features))    
    
    # Save the RMSE
    errors_vs_N.loc[len(errors_vs_N)] = [N, train_error_overfit, validation_error_overfit]

errors_vs_N

If we plot the training and validation error as we add each additional feature, our training error gets lower and lower, and in fact, it's possible to prove with linear algebra that the training error will decrease monotonically.

By contrast, the error on unseen validation data is higher for the models with more parameters, since the lessons learned from these last 20+ features aren't actually useful when applied to unseen data. That is, these models aren't generalizable.

In [ ]:
import plotly.express as px
px.line(errors_vs_N, x = "N", y = ["Training Error", "Validation Error"], width=800, height=400)

<br>

---

### Question 3c

This plot is a useful tool for **model selection**. Describe your observation in the plot above. Can this plot be used to inform the number of features you should use for modeling?

_Type your answer here, replacing this text._


**Bonus:** You may be tempted to iterate through all possible feature combinations. While this may be possible when the dataset contains very few features, a dataset with 48 features has a total of ${48 \choose 1} + {48 \choose 2} + {48 \choose 3} + \cdots + {48 \choose 48}$ combinations. In practice, we often use heuristics like the example above that perform a reasonable amount of comparisons instead of trying out every combination due to computational constraints. 

<br>
<hr style="border: 1px solid #fdb515;" />

## Regularization

Regularization is the formal term that describes the process of limiting a model’s complexity, often with the aim of reducing overfitting and allowing for more generalizable models. Here, we consider Ridge Regression, also termed L2 Regularization (in contrast with LASSO Regression or L1 Regularization), as discussed in Lecture 16. Mathematically speaking, our objective function looks fairly similar to the usual formula for MSE with the addition of the regularization term: 

$$\min_{\theta} \frac{1}{n} || \mathbb{Y} - \mathbb{X}\theta || + \lambda \sum_{j=1}^{d} \theta_j^{2}$$

As an alternative and more realistic example, instead of using only the first N features, we can use various different regularization strengths. For example, for really low regularization strengths (e.g. $\lambda = 10^{-12}$), we get a model that is nearly identical to our linear regression model. This regularization term is so small you may even get a `LinAlgWarning`. Below, we import `Ridge`, which initializes a model similar to `lm.LinearRegression` but applies Ridge Regression.

**Note**: `sklearn` uses `alpha` to represent $\lambda$. The `alpha` parameter passed into `Ridge` represents the regularization parameter, and here, does not refer to the learning rate in the context of gradient descent.

In [ ]:
from sklearn.linear_model import Ridge

regularized_model = Ridge(alpha = 1e-12)
regularized_model.fit(X_train2, Y_train2)
ridge_coefs_low_regularization = regularized_model.coef_

linear_model = linear_model.fit(X_train2, Y_train2)
linear_coefs = linear_model.coef_

# Most of the differences are 0's
(ridge_coefs_low_regularization - linear_coefs).round()

However, if we pick a large regularization strength, e.g. $\lambda = 10^2$, we see that the resulting parameters are much smaller in magnitude. 

In [ ]:
regularized_model = Ridge(alpha = 10**2)
regularized_model.fit(X_train2, Y_train2)
regularized_model_coefs = regularized_model.coef_

# The difference in magnitude are mostly negative, 
# this shows that regularized_model has a smaller coefficient in magnitude
(abs(regularized_model_coefs) - abs(linear_coefs)).round()

### Standard Scaling / Normalization

Recall from the lecture that in order to properly regularize a model, the features should be at the same scale. Otherwise, the model has to spend more of its parameter budget to use "small" features (e.g., lengths in inches) compared to "large" features (e.g., lengths in kilometers).

To do this we can use sklearn's `StandardScaler` [(documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to create a new version of the `DataFrame` where every column has a mean of zero and a standard deviation of 1. 

**Optional:** Can you implement this using a `pandas` function?

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(boston_with_extra_features)
boston_with_extra_features_scaled = pd.DataFrame(ss.transform(boston_with_extra_features), \
                                                 columns = boston_with_extra_features.columns)
boston_with_extra_features_scaled

Let's now regenerate the training and validation sets using this new rescaled dataset.

In [ ]:
np.random.seed(25)
X = boston_with_extra_features_scaled
X_train3, X_validation3, Y_train3, Y_validation3 = train_test_split(X, Y, test_size = 0.10)

Fitting our regularized model with $\lambda = 10^2$ on this scaled data, we now see that our coefficients are of around the same magnitude. This is because all of our features are of around the same magnitude, whereas in the unscaled data, some of the features, like `TAX^2`, were much larger than others.

In [ ]:
regularized_model_standardized = Ridge(alpha = 10**2)
regularized_model_standardized.fit(X_train3, Y_train3)
regularized_model_standardized.coef_

<br>

---

### Question 3d: Finding an Optimum $\lambda$

In the cell below, write code that generates a `DataFrame` with the training and validation error for the range of lambdas given. Make sure you're using the 3rd training and validation sets, which have been rescaled, that is, `X_train3`, `X_validation3`, `Y_train3`, and `Y_validation3`. To lay out the process:

1. Initialize a new `lm.Ridge` model with the regularization hyperparameter set to the new value of $\lambda$.
2. Fit the model to the training set, `X_train3`.
3. Compute the RMSE of the model on the training and validation sets.
4. Add a row to the `DataFrame` containing the value of $\lambda$, training error, and validation error.

**Note**: You should use all 48 features for every single model that you fit, i.e. you're not going to be keeping only the first $N$ features.

**Hint**: It is possible to "append" or add a row to a `DataFrame` by calling `loc`, e.g. `df.loc[len(df)] = [2, 3, 4]` (assuming `df` has 3 columns!).

In [ ]:
error_vs_lambda = pd.DataFrame(columns = ["lambda", "Training Error", "Validation Error"])
range_of_lambda = 10**np.linspace(-5, 4, 40)

for lamb in range_of_lambda:
    regularized_model_lambda = ...
    ...
    train_error_overfit = ...
    holdout_error_overfit = ...
    error_vs_lambda.loc[len(error_vs_lambda)] = ...

error_vs_lambda.head()

In [ ]:
grader.check("q3d")

Below we plot your training and validation set error for the range of lambdas given. You should see a figure where training error decreases as model complexity increases, but the error on the validation set is large for extreme values of lambda and minimized for some intermediate values. Holding your mouse over the training error and validation error lines will display the lambda and value at that point.

Note that on your plot, the **x-axis is the inverse of complexity**! In other words, small lambda models (on the left) are complex, because there is no regularization. That's why the training error is lowest on the left side of the plot, as this is where overfitting occurs.

In [ ]:
px.line(error_vs_lambda, x = "lambda", y = ["Training Error", "Validation Error"], log_x=True, height = 400, width = 800)

<br>

---

### Question 3e

Based on the plot above, which lambda value will you use in your model? Keep in mind that the x-axis in the plot above has a logarithmic scale.

_Type your answer here, replacing this text._

<br>
<hr style="border: 1px solid #fdb515;" />

## REMINDER: Test Set vs. Validation Set (a.k.a. Development Set)

In the plots above, we trained our models on a training set and plotted the resulting RMSE on the training set in blue. We also held out a set of data and plotted the error on this validation set in red, calling it the "validation set error". 

For the example above, we used the validation set to pick a hyperparameter, so we'd also call this set a "development set". These terms are exactly synonymous.

It would not be accurate to call this line the "validation set error" because we did not use this dataset as a test set. While it is true that your code never supplied `X_validation3` or `Y_validation3` to the fit function of the ridge regression models, once you decide to use the validation set to select between different models, different hyperparameters, or different sets of features, then we are not using that dataset as a "test set" but rather a "validation set". That is, since we've used this set for picking lambda, the resulting errors are no longer unbiased predictors of our performance on unseen models -- the true error on an unseen dataset is likely to be somewhat higher than the validation set. After all, we trained 40 models and picked the best one!

In many real-world contexts, model builders will split their data into three sets: training, validation, and test sets, where the test set is *only* ever used once. That is, there are two holdout sets: one used as a development set (for model selection), and one used as a test set (for providing an unbiased estimate of error).

## An Alternate Strategy for Hyper Parameter Selection: K-Fold Cross Validation

Earlier, we used the holdout method for model selection (the holdout method is also sometimes called "simple cross-validation"). Another approach is K-fold cross-validation. This allows us to use more data for training instead of having to set aside some specifically for hyperparameter selection. However, doing so requires more computation resources, as we'll have to fit K models per hyperparameter choice.

In our course, there's really no reason not to use cross-validation. However, in environments where models are very expensive to train (e.g. deep learning), you'll typically prefer using a holdout set (simple cross-validation) rather than K-fold cross-validation.

To emphasize what K-fold cross-validation actually means, we're going to manually carry out the procedure. Recall the approach looks something like the figure below for 4-fold cross-validation:

<img src="cv.png" width=800px>

When we use K-fold cross-validation, rather than using a validation set for model selection, we instead use the training set for model selection. To select between various features, models, or hyperparameters, we split the training set further into multiple temporary train and validation sets (each split is called a "fold", hence K-fold cross-validation). We will use the average validation error across all K folds to make our optimal feature, model, and hyperparameter choices. In this example, we'll only use this procedure for hyperparameter selection, specifically to choose the best lambda.

<br>

---

### Question 4

Scikit-learn has built-in support for cross-validation. However, to better understand how cross-validation works, complete the following function which cross-validates a given model.

1. Use `sklearn`'s `KFold.split` ([documentation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)) function to get 4 splits on the training data. Note that `split` returns the indices of the data for that split.
2. For **each** split:
    1. Select the training and validation rows and columns based on the split indices and features.
    2. Compute the RMSE on the validation split.
    3. Return the average error across all cross-validation splits.


In [ ]:
from sklearn.model_selection import KFold

def compute_CV_error(model, X_train, Y_train):
    '''
    Split the training data into 4 subsets.
    For each subset, 
        - Fit a model holding out that subset.
        - Compute the MSE on that subset (the validation set).
    You should be fitting 4 models in total.
    Return the average MSE of these 4 folds.

    Args:
        model: An sklearn model with fit and predict functions. 
        X_train (DataFrame): Training data.
        Y_train (DataFrame): Label.
    
    Return:
        The average validation MSE for the 4 splits.
    '''
    kf = KFold(n_splits=4)
    validation_errors = []
    
    for train_idx, valid_idx in kf.split(X_train):
        # split the data
        split_X_train, split_X_valid = ..., ...
        split_Y_train, split_Y_valid = ..., ...

        # Fit the model on the training split
        ...
        
        # Compute the RMSE on the validation split
        error = ...


        validation_errors.append(error)
        
    return np.mean(validation_errors)

In [ ]:
grader.check("q4")

<br>

---

### Question 5

Use `compute_CV_error` to add a new column to `error_vs_lambda` which gives the 4-fold cross-validation error for the given choices of $\lambda$ in `range_of_lambda`. `cv_errors` should be a list or an array of cross-validation errors generated by `compute_CV_error` for each tested value of $\lambda$. Again, use the 3rd training and validation sets `X_train3`, `X_validation3`, `Y_train3`, and `Y_validation3`.

In [ ]:
cv_errors = []

...

error_vs_lambda["CV Error"] = cv_errors
error_vs_lambda.head()

In [ ]:
grader.check("q5")

The code below shows the holdout error that we computed in the previous problem as well as the 4-fold cross-validation error. Note that the cross-validation error shows a similar dependency on lambda relative to the holdout error. This is because they are both doing the same thing: trying to estimate the expected error on unseen data drawn from the distribution from which the training set was drawn. 

In other words, this figure compares the holdout method with 4-fold cross-validation.

In [ ]:
px.line(error_vs_lambda, x = "lambda", y = ["Validation Error", "CV Error"], 
        log_x=True, height = 400, width = 800)

<br/><br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Otto congratulates you for finishing Lab 8!

<img src="otto.jpeg" alt="Paris" class="center" width="500">

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)